# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [13]:
from tchub.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [14]:
config = AttrDict(
    use_wandb = False, # Whether to use or not wandb for experiment tracking
    wandb_group = 'embeddings', # Whether to group this run in a wandb group
    wandb_entity = 'pacmel',
    wandb_project = 'tchub',
    enc_artifact = 'pacmel/tchub/mvp:v5', # Name:version of the encoder artifact
    input_ar = None,
    cpu = False
)

## Run

In [15]:
run = wandb.init(entity=config.wandb_entity,
                    project=config.wandb_project if config.use_wandb else 'work-nbs', 
                    group=config.wandb_group,
                    job_type='embeddings', 
                    mode='online' if config.use_wandb else 'disabled',
                    anonymous = 'never' if config.use_wandb else 'must',
                    config=config,
                    #id = 'embeddingsProvider',
                    resume='allow')

In [16]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [17]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [18]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()
enc_learner

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [19]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_valid = artifacts_gettr(enc_run.config['valid_artifact'], type='dataset')
enc_artifact_train.name, enc_artifact_valid.name

('steamflow:v0', 'steamflow:v0')

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [20]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_valid.entity}/{enc_artifact_valid.project}/{enc_artifact_valid.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'steamflow:v0'

In [21]:
df = input_ar.to_df()
df

,steam flow
1970-01-01 00:00:00,9.302970
1970-01-01 00:00:03,9.662621
1970-01-01 00:00:06,10.990955
1970-01-01 00:00:09,12.430107
1970-01-01 00:00:12,13.681666
...,...
1970-01-01 07:59:45,7.898815
1970-01-01 07:59:48,7.729076
1970-01-01 07:59:51,7.856854
1970-01-01 07:59:54,7.799764


In [22]:
df.shape

(9600, 1)

In [23]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(8961, 1, 640)

In [24]:
%%time
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

/home/victor/env/lib/python3.8/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


CPU times: user 4.71 s, sys: 127 ms, total: 4.84 s
Wall time: 4.94 s


In [25]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [26]:
run.finish()